In [101]:
# 네이버 증권에서 거래량 천만주 이상인 종목 크롤링

from telbot import send_msg
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [122]:

# 상한가 종목

url0= 'https://finance.naver.com/sise/sise_upper.naver'

raw_data = requests.get(url0)
data = bs(raw_data.content, 'html.parser')
table = data.find_all('table')
#print(table[1])
data = []

exclude_keywords = ["KODEX", "ARIRANG", "TIGER", "인버스", "SOL", "ACE"]

for row in table[1].find_all('tr'):
    row_data = []
    for cell in row.find_all(['td', 'th']):
        row_data.append(cell.get_text().strip())
    if len(row_data) > 1:
        if not any(keyword in row_data[1] for keyword in exclude_keywords):
            data.append(row_data)

column_name = data[0]
df1 = pd.DataFrame(data, columns=column_name)
df1 = df1[1:].drop(columns = ['N'])


data2 = []
for row in table[2].find_all('tr'):
    row_data = []
    for cell in row.find_all(['td', 'th']):
        row_data.append(cell.get_text().strip())
    if len(row_data) > 1:
        if not any(keyword in row_data[1] for keyword in exclude_keywords):
            data2.append(row_data)
column_name = data2[0]
df2 = pd.DataFrame(data2, columns=column_name)
df2 = df2[1:].drop(columns = ['N'])
merged_df = pd.concat([df1,df2])
merged_df.head(20)

,연속,누적,종목명,현재가,전일비,등락률,거래량,시가,고가,저가,PER
1,1,1,솔루스첨단소재,"16,970","3,910",+29.94%,"15,567,431","15,250","16,970","15,040",7.90
2,1,1,솔루스첨단소재2우B,"6,490","1,495",+29.93%,"2,094,198","6,350","6,490","5,630",3.02
3,1,3,흥국화재우,"9,780","2,250",+29.88%,"172,476","7,430","9,780","7,430",2.97
1,1,1,와이씨켐,"18,070","4,170",+30.00%,"4,824,709","13,910","18,070","13,530",-30.17
2,1,1,디에이테크놀로지,462,106,+29.78%,"36,785,166",430,462,399,-1.77
3,1,1,아이톡시,"1,727",398,+29.95%,"4,346,517","1,700","1,727","1,600",172.70


In [123]:
# 코스피 거래량 상위

url1 = "https://finance.naver.com/sise/sise_quant.naver"

raw_data = requests.get(url1)
data = bs(raw_data.content, 'html.parser')
table = data.find_all('table')
#print(table[1])
data = []

exclude_keywords = ["KODEX", "ARIRANG", "TIGER", "인버스", "SOL", "ACE"]

for row in table[1].find_all('tr'):
    row_data = []
    for cell in row.find_all(['td', 'th']):
        row_data.append(cell.get_text().strip())
    if len(row_data) > 1:
        if not any(keyword in row_data[1] for keyword in exclude_keywords):
            data.append(row_data)

#print(data)
column_name = data[0]

df = pd.DataFrame(data, columns=column_name)
df = df[1:].drop(columns = ['N'])
numeric_columns = ['현재가', '거래량', '거래대금', '매수호가', '매도호가', '시가총액']

df[numeric_columns] = df[numeric_columns].apply(lambda x: x.str.replace(',', ''))

# Convert columns to numeric if necessary
df[numeric_columns] = df[numeric_columns].astype(int)

df = df[["종목명", "현재가", "등락률", "거래량"]]
df_filtered = df[df['거래량'] > 10_000_000]
df_filtered.head(20)

,종목명,현재가,등락률,거래량
1,흥아해운,2915,+14.09%,43499052
2,대한해운,1931,+2.44%,32035890
3,에이프로젠,1510,-11.23%,17460246
4,삼성전자,79800,-0.13%,17220616
5,솔루스첨단소재,16970,+29.94%,15567431
6,이수페타시스,43750,+11.75%,14455176
7,아센디오,1356,+2.34%,14224456
8,화천기계,6700,-23.69%,12412220
9,동방,2250,+6.89%,12238910


In [124]:
# 코스닥 거래량 상위

url2 = "https://finance.naver.com/sise/sise_quant.naver?sosok=1"

raw_data = requests.get(url2)

html = raw_data.content.decode('euc-kr','replace').encode('utf-8','replace')


data = bs(html, 'html.parser')
table = data.find_all('table')
#print(table[1])
data = []

exclude_keywords = ["호스팩", "KODEX", "ARIRANG", "TIGER", "인버스", "SOL", "ACE"]

for row in table[1].find_all('tr'):
    row_data = []
    for cell in row.find_all(['td', 'th']):
        row_data.append(cell.get_text().strip())
    if len(row_data) > 1:
        if not any(keyword in row_data[1] for keyword in exclude_keywords):
            data.append(row_data)

column_name = data[0]

df = pd.DataFrame(data, columns=column_name)
df = df[1:].drop(columns = ['N'])
numeric_columns = ['현재가', '거래량', '거래대금', '매수호가', '매도호가', '시가총액']

df[numeric_columns] = df[numeric_columns].apply(lambda x: x.str.replace(',', ''))

# Convert columns to numeric if necessary
df[numeric_columns] = df[numeric_columns].astype(int)
df = df[["종목명", "현재가", "등락률", "거래량"]]
df_filtered = df[df['거래량'] > 10_000_000]
df_filtered.head(20)

,종목명,현재가,등락률,거래량
1,골든센츄리,114,+4.59%,89208893
2,케이사인,1487,-7.87%,43010293
3,디에이테크놀로지,462,+29.78%,36785166
4,코닉오토메이션,4000,+28.00%,30722395
5,이스트아시아홀딩스,95,-2.06%,25191927
6,SM C&C,1862,+9.27%,18200293
7,샌즈랩,11260,+9.64%,16617976
8,오가닉티코스메틱,82,+1.23%,15872031
9,리튬포어스,7680,+12.78%,15705871
10,하나마이크론,28150,+10.39%,15618095
